In [1]:
import pandas as pd
import numpy as np
import ast

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv(r"../output/DEFAULT_FINAL.csv")

In [3]:
# List of metric names
metric_names = ["lift",
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "roc_auc",
    "area_under_pr",
    "r2_score",
    "rmse"
]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset["output_metrics"] = dataset["output_metrics"].apply(ast.literal_eval)
dataset["run_time"] = dataset["run_time"].apply(lambda x: np.round(x/60, 2))
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset["output_metrics"].apply(
        lambda x: x.get(metric) if isinstance(x, dict) else np.nan
    )
    dataset[f"{metric}_std"] = dataset[metric].apply(
        lambda x: np.std(x) if isinstance(x, list) else np.nan
    )
    dataset[metric] = dataset[metric].apply(
        lambda x: np.average(x) if isinstance(x, list) else np.nan
    )

base_cols   = [
    "dataset",
    "model",
    "run_time",
    "eval_metric",
    "best_score",
    "score_std",
    "output_metrics"]

base_cols += [i for i in metric_names]
base_cols += [i+"_std" for i in metric_names]

dataset = dataset[base_cols]

In [4]:
tomax = {
    "mse": False,
    "rmse": False,
    "accuracy": True,
    "recall": True,
    "precision": True,
    "roc_auc": True,
    "area_under_pr": True,
    "lift": True,
    "f1": True,
    "r2_score": True,
}

# Define a function to select the first row based on whether to maximize or minimize the "best_score"
def select_first_row(group):
    metric = group["eval_metric"].iloc[0]
    ascending = not tomax.get(metric, True)  # If metric not in tomax, assume True (maximize)
    return group.sort_values(by="best_score", ascending=ascending).iloc[0]


# Apply the function to each group
filtered_df = dataset.groupby(["dataset", "model", "eval_metric"]).apply(select_first_row)
# Reset the index to get a new DataFrame
filtered_df = filtered_df.reset_index(drop=True)
# Find the rows that maximize the specified metric for each dataset


# Sort the DataFrame based on whether the metric is to be maximized or not
filtered_df["ascending"] = filtered_df["eval_metric"].map({k:not v for k,v in tomax.items()})  # Create a new column for ascending order
dfmax = filtered_df.loc[filtered_df["ascending"] == False].sort_values(by=["dataset", "eval_metric", "best_score"], ascending=[False, False, False])
dfmin = filtered_df.loc[filtered_df["ascending"] == True].sort_values(by=["dataset", "eval_metric", "best_score"], ascending=[False, False, True])

best_df = pd.concat([dfmax,dfmin])
best_df.drop(columns=["ascending"], inplace=True)
best_df.head(100)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
57,titanic,resnet,0.53,lift,3.046058,0.362876,"{'recall': [0.9104477611940298, 0.859375, 0.83...",3.046058,0.900270,NaN,0.938278,0.869890,0.935903,0.956824,0.940612,NaN,NaN,0.362876,0.025761,NaN,0.011199,0.054636,0.025431,0.014577,0.021226,NaN,NaN
51,titanic,catboost,0.53,lift,2.544501,0.089689,"{'recall': [0.7391304347826086, 0.764705882352...",2.544501,0.769090,NaN,0.836099,0.713384,0.836864,0.881115,0.857053,NaN,NaN,0.089689,0.041024,NaN,0.026399,0.050898,0.047627,0.017801,0.016332,NaN,NaN
60,titanic,tabtransformer,0.67,lift,2.544055,0.076238,"{'recall': [0.7391304347826086, 0.764705882352...",2.544055,0.740484,NaN,0.800207,0.742626,0.738722,0.843048,0.829274,NaN,NaN,0.076238,0.019533,NaN,0.015113,0.020595,0.024858,0.014490,0.020611,NaN,NaN
50,titanic,autoint,0.30,lift,2.513705,0.085630,"{'recall': [0.8115942028985508, 0.794117647058...",2.513705,0.740424,NaN,0.791243,0.777536,0.707235,0.861729,0.829969,NaN,NaN,0.085630,0.028250,NaN,0.019285,0.041999,0.020787,0.016240,0.014752,NaN,NaN
54,titanic,gandalf,0.39,lift,2.513705,0.085630,"{'recall': [0.8260869565217391, 0.779411764705...",2.513705,0.763372,NaN,0.818134,0.768627,0.759127,0.868772,0.847891,NaN,NaN,0.085630,0.050296,NaN,0.034566,0.065001,0.039928,0.023869,0.021617,NaN,NaN
61,titanic,xgb,0.01,lift,2.482910,0.119197,"{'recall': [0.7681159420289855, 0.794117647058...",2.482910,0.777888,NaN,0.838365,0.739685,0.821129,0.874329,0.847345,NaN,NaN,0.119197,0.027343,NaN,0.016595,0.040329,0.016061,0.020416,0.029427,NaN,NaN
56,titanic,mlp,0.01,lift,2.452390,0.143021,"{'recall': [0.6666666666666666, 0.632352941176...",2.452390,0.711148,NaN,0.797979,0.649105,0.787613,0.834473,0.799974,NaN,NaN,0.143021,0.023124,NaN,0.013769,0.034855,0.018000,0.012359,0.011375,NaN,NaN
58,titanic,s1dcnn,0.03,lift,2.267890,0.326553,"{'recall': [0.0, 0.029411764705882353, 0.01470...",2.267890,0.022857,NaN,0.619534,0.011722,0.500000,0.832290,0.771237,NaN,NaN,0.326553,0.021381,NaN,0.005021,0.010994,0.447214,0.024745,0.065385,NaN,NaN
53,titanic,fttransformer,0.82,lift,2.205471,0.721286,"{'recall': [0.782608695652174, 0.0, 0.69117647...",2.205471,0.611493,NaN,0.777685,0.615857,0.608281,0.788943,0.755340,NaN,NaN,0.721286,0.308474,NaN,0.085766,0.311032,0.308039,0.169059,0.195174,NaN,NaN
55,titanic,gate,4.35,lift,2.175122,0.411114,"{'recall': [0.7536231884057971, 0.75, 0.705882...",2.175122,0.547951,NaN,0.731649,0.546249,0.551259,0.780041,0.724321,NaN,NaN,0.411114,0.283898,NaN,0.077389,0.286004,0.284270,0.107188,0.133788,NaN,NaN
